# Question Answering System by Retrieval Augmented Generation

### *Guide*

Anton Antonov    
September 2024  

-----

## Introduction

This notebook shows how to import an LLM-computed vector database and then LLM-generate from it responses to certain queries.

-------

## Setup

Packages used below:

In [26]:
use Data::Importers;
use LLM::Functions;
use XDG::BaseDirectory :terms;

use LLM::RetrievalAugmentedGeneration;
use LLM::RetrievalAugmentedGeneration::VectorDatabase;

use Data::Reshapers;
use Data::Summarizers;
use Math::Nearest;
use Math::DistanceFunctions;
use Statistics::OutlierIdentifiers;

A special LLM configuration:

In [33]:
my $conf4o = llm-configuration('ChatGPT', model => 'gpt-4o', max-tokens => 4096, temperature => 0.4);
$conf4o.Hash.elems

24

-----

## Import Vector Database

In this section we import the vector database and do certain basic statistics over it.

Here we make and empty vector database object:

In [2]:
my $vdbObj = LLM::RetrievalAugmentedGeneration::VectorDatabase.new();

VectorDatabase(:id("8de131f4-c0e5-4cc5-be44-dcdd1d3d8344"), :name(""), :elements(0), :sources(0))

We can see the available _pre-computed_ vector databases with `vector-database-objects`:

In [3]:
vector-database-objects».basename

[SemSe-266b20ca-d917-4ac0-9b0a-7c420625666c.json]

Here using we form a file path for a previously computed (and exported) vector database using [`$XDG_DATA_HOME`](https://specifications.freedesktop.org/basedir-spec/latest/index.html):

In [4]:
# The sub-directory
my $dirname = data-home.Str ~ '/raku/LLM/SemanticSearchIndex';

# The exported vector database base file name
my $basename = 'SemSe-266b20ca-d917-4ac0-9b0a-7c420625666c.json';

# Corresponding IO:Path object
my $file = IO::Path.new(:$dirname, :$basename);

# Check for existence
$file.f

True

Import the vector database:

In [6]:
my $tstart = now;

$vdbObj.import($file);

my $tend = now;

say "Import time { $tend - $tstart } seconds.";

Import time 4.168099464 seconds.


Here is the vector database object's _gist_:

In [7]:
say $vdbObj;

VectorDatabase(:id("266b20ca-d917-4ac0-9b0a-7c420625666c"), :name("No833"), :elements(441), :sources(442))


Show text chunks sample:

In [8]:
#% html
$vdbObj.text-chunks.pick(3) ==> to-html()

428.0,"We recognize the culture, the endowment, the contribution to American society. And I wish the democratic party were doing this. They're right there. Go speak to them, stop spinning on them and shitting on them and pissing on them. But somehow we've got this sort of NAFTA coked up on you. You're not going to be able to do that. You're not going to Go speak to them. Stop spinning on them and shitting on them and pissing on them. But somehow we've got this sort of NAFTA, coked up on NAFTA intellectual elite that says, Hey, we helped people in Mexico."
264.0,"that you ask a difficult question as opposed to there is this compulsion inside of me, I'm dragged forward to go, well, what I mean, and, and throw this sort of lifebuoy to them, uh, because sitting with that discomfort. So that is by far, I think the most valid criticism. Now, one of the things that nobody gets to see is how hard it is for people to achieve the things that they achieve. So apart from in sports,"
171.0,"it's, it's an absolutely diabolical situation. So the point that I find interesting is not, although it is, what is the motivation of the criticizer is interesting to me, but what is the response of the criticized is much more interesting to me. What are the options? I'm actually just interested. What do you see as the options that are possible? Because I can't, this is a problem that I can't solve."


Show vector database dimensions:

In [9]:
$vdbObj.database.&dimensions

(441 1536)

Show vectors' norms:

In [10]:
$vdbObj.database.pick(3).deepmap({ norm($_.value) })

(1.0000000265764895 0.9999999336431316 0.9999999738155717)

Here is a summary over all vectors:

In [12]:
sink records-summary($vdbObj.database.values.map({ norm($_) }))

+------------------------------+
| numerical                    |
+------------------------------+
| Max    => 1.0000001001691734 |
| Median => 1.000000012338632  |
| Min    => 0.9999999127751137 |
| 3rd-Qu => 1.0000000370665867 |
| 1st-Qu => 0.9999999894227217 |
| Mean   => 1.0000000117763133 |
+------------------------------+


------

## Themes found in the text chunks

The vector database object has the attributes:
- `database` which is a `Map` of labels (identifiers) to LLM embedding vectors
- `text-chunks` which is a `Map` of labels to text chunks that correspond to the embedding vectors

(The keys of `database` and `text-chunks` are the same.)


By examining the LLM-extracted themes from text chunks of the imported vector database, 
we see that the discussion they came from is [fairy eclectic](https://www.youtube.com/watch?v=PYRYXhU4kxM), [CWv1]:

In [13]:
#% html
my $res = llm-synthesize([
    llm-prompt("ThemeTableJSON")(
        $vdbObj.text-chunks.sort(*.key)».value.join("\n"), 'text', 15, 
    )
    ],
    e => $conf4o,
    form => sub-parser('JSON'):drop
);

$res ==> to-html(field-names => <theme content>, align => 'left')

theme,content
Introduction,Introduction of the guest and overview of topics to be discussed.
Eric's Predictions,Discussion about the 2024 presidential election and potential outcomes.
Media and Gaslighting,Analysis of media behavior and global gaslighting.
International Order,Discussion on international agreements and political structures.
Trump's Presidency,Impact and implications of Donald Trump's presidency.
Election Dynamics,How democracy and elections are influenced by international order.
Censorship and Control,Role of censorship and media in shaping public opinion.
AI and Technology,"Discussion on AI, technology, and their societal impact."
String Theory,Critique of string theory and its dominance in physics.
Criticism and Public Perception,Handling criticism and its impact on public figures.


**Remark:** It is instructive to compare the extracted themes with the list video segments given in [CWv1]. 

-----

## Nearest neighbors finding

Here is a query:

In [15]:
my $query = 'What is the state of string theory?';

What is the state of string theory?

Here we find the labels of the vectors (and text chunks) that _considered_ nearest neighbors of certain query:

In [16]:
my @nnLabels = $vdbObj.nearest($query, 10, prop => <label distance>, distance-function => &euclidean-distance)

[(126.0 0.9066815079532051) (139.0 0.9301226226546818) (121.0 0.9664632700051056) (112.0 0.9696340383729329) (162.0 1.008971602214595) (138.0 1.020272389983001) (127.0 1.0270789467175634) (141.0 1.074852800470846) (137.0 1.08715933633032) (125.0 1.0941371063568315)]

Here we make the corresponding dataset that included the text chunks from the vector database:

In [17]:
my @dsScores = @nnLabels.map({
    %( label => $_[0], distance => $_[1], text => $vdbObj.text-chunks{$_[0]} )
});

@dsScores.&dimensions

(10 3)

Here we show the dataset:

In [18]:
#% html
@dsScores ==> to-html(field-names => <distance label text>, align => 'left');

distance,label,text
0.9066815079532051,126.0,"But is somebody at the forefront of string theory? Absolutely. And he said, quote, I can tell you with absolute certainty, string theory is not the theory of the real world. I can tell you that 100%. My strong feelings are exactly that string theory is definitely not the theory of the real world. I can tell you that 100%. My strong feelings are exactly that string theory is definitely not the theory of the real world. Is that taking it out of context? Is that him framing it somewhere else? Or does that encapsulate the fact"
0.9301226226546818,139.0,"And I saw a tweet saying that somebody had been to a string theory convention and had asked the question, what is string theory? And the best string theorists on the planet came up with the answer, we kind of don't know what string theory is. And the other answer is whatever it is that we're doing. Whatever it is that the string theory community is doing. Even if they did something that had nothing to do with string theory,"
0.9664632700051056,121.0,"That is not shiny. That is saying everything else is crap and dangerous. In other words, it's string theory can't sell itself as physics. By any telling of the story, string theory is the most failed theory in the history of physics. If you look at the number of papers, the amount of money, the number of people, the number of PhDs, number of conferences, achievements in physics proper per investment or size of effort. It is the most failed theory in the history of physics and the way in which it survives is by hunting and destroying its enemies and making its"
0.9696340383729329,112.0,"We don't know whether that we're talking about the stagnation of theoretical physics or just nuclear physics. You're okay with speculating. Let's speculate. I'll do the decision tree. One possibility is that they're simply saying that they made nuclear physics very, very difficult to do. And that has to do with not very sexy physics, the physics of protons and neutrons in nuclear. So that branch exists. The other branch says, um, we used string theory to cock block actual progress in theoretical physics and derailed an entire field, at least"
1.008971602214595,162.0,"Everybody in the community reads it and many people pretend that they don't because it's very critical of string theory, but he's very, very good. Then he writes a book like this. Nobody saw it coming. Then he comes up with two theories, both of which I of string theory, but he's very, very good. Then he writes a book like this, nobody saw it coming. Then he comes up with two theories, both of which I think are wrong, but are really, really clever about the nature of the strong force, what would be called weaker hypercharge"
1.020272389983001,138.0,"and removed with extreme prejudice. It's anti-science. So I don't know much or anything really about the inner workings of string theory, but Sabine Hossenfeld has been on the show, Brian Greene's been on the show, Sean Carroll's been on the show. Oh, let's get them, all of them."
1.0270789467175634,127.0,"that he thinks string theory is a dead end that doesn't describe the world? He's playing a game that I would, I would say is Logomachy, an argument over words, where he says that big S string theory is not the theory of the real world, which is the theory that was used to destroy all of its competitors and that little S string theory exists. I don't, this is basically the attempt, uh, to take a school massacre and plead to a parking ticket. And no, I think that the prosecution should decline the offer from the good Dr. Suskin and say, no, no, no, you have 40 years of the destruction of your colleagues to answer for you've chosen to be, um, words, family, an asshole,"
1.074852800470846,141.0,"No, Mr. Smart, I don't believe that either. Two Cub Scouts with slingshots. So, this is a very old pattern. Yeah. Is this too far gone for string theory now? Is it the mask is beginning to slip to the point wher

-----

## Nearest neighbors finding (low-level)

In this section we show how to find the elements of the vector database that are _considered_ nearest neighbors to the query vector. 
We use "low-level" computations for didactic purposes.
Same or similar results can be obtained by using the method `nearest` of the vector database object.

Here is its vector embedding (with the same LLM that was used to compute the vector database):

In [19]:
my @query-vector = |llm-embedding($query, llm-evaluator => $vdbObj.llm-configuration).head;

@query-vector.elems

1536

For each vector in the database find its distance to the query vector:

In [20]:
my @dsScores =
        $vdbObj.database.map({ %(
            label => $_.key,
            distance => euclidean-distance($_.value, @query-vector),
            text => $vdbObj.text-chunks{$_.key}
        ) }).Array;

@dsScores.&dimensions

(441 3)

Sort in ascending order:

In [21]:
@dsScores .= sort({ $_<distance> });

@dsScores.map(*<distance>).head(6)

(0.9066815079532051 0.9301226226546818 0.9664632700051056 0.9696340383729329 1.008971602214595 1.020272389983001)

Show the text chunks closest to the query:

In [22]:
#% html
@dsScores.head(12) ==> to-html(field-names => <distance label text>, align => 'left');


distance,label,text
0.9066815079532051,126.0,"But is somebody at the forefront of string theory? Absolutely. And he said, quote, I can tell you with absolute certainty, string theory is not the theory of the real world. I can tell you that 100%. My strong feelings are exactly that string theory is definitely not the theory of the real world. I can tell you that 100%. My strong feelings are exactly that string theory is definitely not the theory of the real world. Is that taking it out of context? Is that him framing it somewhere else? Or does that encapsulate the fact"
0.9301226226546818,139.0,"And I saw a tweet saying that somebody had been to a string theory convention and had asked the question, what is string theory? And the best string theorists on the planet came up with the answer, we kind of don't know what string theory is. And the other answer is whatever it is that we're doing. Whatever it is that the string theory community is doing. Even if they did something that had nothing to do with string theory,"
0.9664632700051056,121.0,"That is not shiny. That is saying everything else is crap and dangerous. In other words, it's string theory can't sell itself as physics. By any telling of the story, string theory is the most failed theory in the history of physics. If you look at the number of papers, the amount of money, the number of people, the number of PhDs, number of conferences, achievements in physics proper per investment or size of effort. It is the most failed theory in the history of physics and the way in which it survives is by hunting and destroying its enemies and making its"
0.9696340383729329,112.0,"We don't know whether that we're talking about the stagnation of theoretical physics or just nuclear physics. You're okay with speculating. Let's speculate. I'll do the decision tree. One possibility is that they're simply saying that they made nuclear physics very, very difficult to do. And that has to do with not very sexy physics, the physics of protons and neutrons in nuclear. So that branch exists. The other branch says, um, we used string theory to cock block actual progress in theoretical physics and derailed an entire field, at least"
1.008971602214595,162.0,"Everybody in the community reads it and many people pretend that they don't because it's very critical of string theory, but he's very, very good. Then he writes a book like this. Nobody saw it coming. Then he comes up with two theories, both of which I of string theory, but he's very, very good. Then he writes a book like this, nobody saw it coming. Then he comes up with two theories, both of which I think are wrong, but are really, really clever about the nature of the strong force, what would be called weaker hypercharge"
1.020272389983001,138.0,"and removed with extreme prejudice. It's anti-science. So I don't know much or anything really about the inner workings of string theory, but Sabine Hossenfeld has been on the show, Brian Greene's been on the show, Sean Carroll's been on the show. Oh, let's get them, all of them."
1.0270789467175634,127.0,"that he thinks string theory is a dead end that doesn't describe the world? He's playing a game that I would, I would say is Logomachy, an argument over words, where he says that big S string theory is not the theory of the real world, which is the theory that was used to destroy all of its competitors and that little S string theory exists. I don't, this is basically the attempt, uh, to take a school massacre and plead to a parking ticket. And no, I think that the prosecution should decline the offer from the good Dr. Suskin and say, no, no, no, you have 40 years of the destruction of your colleagues to answer for you've chosen to be, um, words, family, an asshole,"
1.074852800470846,141.0,"No, Mr. Smart, I don't believe that either. Two Cub Scouts with slingshots. So, this is a very old pattern. Yeah. Is this too far gone for string theory now? Is it the mask is beginning to slip to the point wher

Show the nearest neighbors scores (with the chosen distance function):

In [23]:
text-list-plot(@dsScores.map(*<distance>), width => 100, height => 16)

+----+-------------------+-------------------+-------------------+-------------------+-------------+      
|                                                                                                  |      
+                                                                                         ****     +  1.40
|                                                              ****************************        |      
|                             **********************************                                   |      
+                  ************                                                                    +  1.30
|            *******                                                                               |      
+         ****                                                                                     +  1.20
|       ***                                                                                        |      
+      **                            

From the plot we can see that there are clear outliers. Here are find outliers' positions, [AA1, AAp8]:

In [24]:
my @pos = outlier-identifier(@dsScores.map(*<distance>), identifier => (&bottom-outliers o &quartile-identifier-parameters));

@pos.max

74

**Remark:** We show only the max outlier position since the distances are sorted in ascending order.

**Remark:** The outlier identifiers `&hampel-identifier-parameters` and `&splus-quartile-identifier-parameters` give 84 and 22 outliers, respectively.

The text chunks correspond to the found outliers are considered most relevant to the query and can be used to form prompt LLM answers for the query.

-----

## Answer based on nearest neighbors

Generate an answer:

In [34]:
#% markdown
my $answer = llm-synthesize([
    'Come up with a narration answering this question:',
    $query,
    "using these discussion statements:",
    @dsScores.head(40).map(*<text>).join("\n")
    ],
    e => $conf4o
);

$answer

The state of string theory is a complex and contentious topic within the scientific community. While it once held the promise of being the ultimate theory of everything, unifying all fundamental forces of nature, its current status is far from settled. 

On one hand, there are prominent figures in the field who have unequivocally stated that string theory is not the theory of the real world. As one leading theorist put it, "I can tell you with absolute certainty, string theory is not the theory of the real world. I can tell you that 100%." This strong sentiment reflects a growing disillusionment among some physicists who have invested decades into the theory, only to find it lacking in empirical validation.

Moreover, the internal discourse within the string theory community reveals a sense of uncertainty and introspection. A tweet from a recent string theory convention highlighted that even the best minds in the field struggle to define what string theory actually is. The prevailing sentiment was that string theory could be described as "whatever it is that we're doing," even if that includes activities tangentially related to the original aims of the theory.

Critics argue that string theory has become the most failed theory in the history of physics, considering the vast resources—papers, money, PhDs, and conferences—dedicated to it without yielding significant achievements in physics proper. Some go as far as to claim that string theory has stifled progress in theoretical physics, diverting attention and resources away from potentially more fruitful avenues of research.

The sociology of the string theory community has also come under scrutiny. Accusations of it being an insular and self-preserving group, hostile to alternative theories, are not uncommon. This has led to a polarized environment where critics and proponents rarely share the same stage, reminiscent of a divide between professional wrestling and mixed martial arts—one being a spectacle, the other a genuine sport.

Despite these criticisms, there are still those who find value in the mathematical structures and mechanisms inspired by string theory. Some physicists appreciate the elegance and potential insights that string theory offers, even if it falls short of being the ultimate theory of everything.

In summary, the state of string theory is one of profound debate and reflection. While it has not delivered on its initial promises and faces significant criticism, it continues to inspire and challenge the boundaries of theoretical physics. The future of string theory remains uncertain, with its ultimate fate likely to be determined by whether it can evolve and adapt to address its current shortcomings.

-------

## References

### Articles

[AA1] Anton Antonov, 
["Outlier detection in a list of numbers"](https://rakuforprediction.wordpress.com/2022/05/29/outlier-detection-in-a-list-of-numbers/),
(2022),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

### Packages

[AAp1] Anton Antonov,
[WWW::OpenAI Raku package](https://github.com/antononcube/Raku-WWW-OpenAI),
(2023),
[GitHub/antononcube](https://github.com/antononcube).

[AAp2] Anton Antonov,
[WWW::PaLM Raku package](https://github.com/antononcube/Raku-WWW-PaLM),
(2023),
[GitHub/antononcube](https://github.com/antononcube).

[AAp3] Anton Antonov,
[LLM::Functions Raku package](https://github.com/antononcube/Raku-LLM-Functions),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp4] Anton Antonov,
[LLM::Prompts Raku package](https://github.com/antononcube/Raku-LLM-Prompts),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp5] Anton Antonov,
[ML::FindTextualAnswer Raku package](https://github.com/antononcube/Raku-ML-FindTextualAnswer),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp6] Anton Antonov,
[Math::Nearest Raku package](https://github.com/antononcube/Raku-Math-Nearest),
(2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp7] Anton Antonov,
[Math::DistanceFunctions Raku package](https://github.com/antononcube/Raku-Math-DistanceFunctions),
(2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp8] Anton Antonov,
[Statistics::OutlierIdentifiers Raku package](https://github.com/antononcube/Raku-Statistics-OutlierIdentifiers),
(2022),
[GitHub/antononcube](https://github.com/antononcube).

## Videos

[CWv1] Chris Williamson,
["Eric Weinstein - Are We On The Brink Of A Revolution? (4K)"](https://www.youtube.com/watch?v=PYRYXhU4kxM),
(2024),
[YouTube/@ChrisWillx](https://www.youtube.com/@ChrisWillx).   
([transcript](https://podscripts.co/podcasts/modern-wisdom/833-eric-weinstein-are-we-on-the-brink-of-a-revolution).)